# Notebook - Raw Data Process
No need to execute this notebook as the dataset is converted and produced in dataset folder.

## Import Modules

In [1]:
import cv2
import json
import numpy as np
import pandas as pd
from netCDF4 import Dataset
from netCDF4 import num2date
from haversine import inverse_haversine, Direction, Unit, haversine
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Feature Processing & Engineering - Sequence Features

### A. Surrogate dataset

In [2]:
# 1. Import source file
wild_fire_surrogate = Dataset('../raw_dataset/WILDFIRE_SURROGATE.nc')

# 2. Get seq_dim for global time adjustments - Monthly Frequency
surrogate_seq_dim = np.array(num2date(wild_fire_surrogate["time"],wild_fire_surrogate["time"].units))

# === Input Features
fuel_load_cwdc = np.array(wild_fire_surrogate["CWDC"])
fuel_load_deadcrootc = np.array(wild_fire_surrogate["DEADCROOTC"])
fuel_wetness = np.array(wild_fire_surrogate["SOILWATER_10CM"])
fuel_temperature = np.array(wild_fire_surrogate["TSOI_10CM"])
climate_wind = np.array(wild_fire_surrogate["WIND"])
climate_tbot = np.array(wild_fire_surrogate["TBOT"])
climate_rh2m = np.array(wild_fire_surrogate["RH2M"])
climate_rain = np.array(wild_fire_surrogate["RAIN"])
tree_coverage = np.array(wild_fire_surrogate["PCT_NAT_PFT"])
# === Output Variable
burned_area = np.array(wild_fire_surrogate["FAREA_BURNED"])

# === size
sorrogate_size = [wild_fire_surrogate["FAREA_BURNED"][0].shape[0],wild_fire_surrogate["FAREA_BURNED"][0].shape[1]]

# print("#--> Start Time Frame: ",surrogate_seq_dim[0])
# print("#--> End Time Frame: ",surrogate_seq_dim[-1])
# print("#--> Time Frame Frequency: ",surrogate_seq_dim[1] - surrogate_seq_dim[0])
# print("#--> Time Frame Length: ",len(surrogate_seq_dim))

In [3]:
def feature_transformation(feature, save_path, name):
    feature = np.where(feature == feature.max(), 0, feature)
    np.save(file = save_path + name + ".npy",arr = feature)
    print("Feature saved: ", name)

In [4]:
# === define saving route
save_path = "../dataset/"

feature_transformation(fuel_load_cwdc, save_path, "fuel_load_cwdc")
feature_transformation(fuel_load_deadcrootc, save_path, "fuel_load_deadcrootc")
feature_transformation(fuel_wetness, save_path, "fuel_wetness")
feature_transformation(fuel_temperature, save_path, "fuel_temperature")
feature_transformation(climate_wind, save_path, "climate_wind")
feature_transformation(climate_tbot, save_path, "climate_tbot")
feature_transformation(climate_rh2m, save_path, "climate_rh2m")
feature_transformation(climate_rain, save_path, "climate_rain")
feature_transformation(tree_coverage, save_path, "tree_coverage")
feature_transformation(burned_area, save_path, "percent_burned_area")

Feature saved:  fuel_load_cwdc
Feature saved:  fuel_load_deadcrootc
Feature saved:  fuel_wetness
Feature saved:  fuel_temperature
Feature saved:  climate_wind
Feature saved:  climate_tbot
Feature saved:  climate_rh2m
Feature saved:  climate_rain
Feature saved:  tree_coverage
Feature saved:  percent_burned_area


### Burned Area Computation & Save

In [5]:
# === compute the grid area

# Mapping the (96,144) to standard lat and lon systems to find out the true lat and lon for each grid
lat_angle = 90
lon_ange = 180

lat_list = np.arange(-lat_angle,lat_angle,lat_angle*2/97)
lon_list = np.arange(-lon_ange,lon_ange,lon_ange*2/145)
lat_list_for_lon_compute = np.arange(-lat_angle,lat_angle,lat_angle*2/96)
print(len(lat_list),len(lon_list),len(lat_list_for_lon_compute)) # 97, 145

# Create an x_distance list (x_distance for the grid will change with latitude change)
x_distance = []
for lon in range(len(lon_list) - 1):
    temp_list = []
    for lat in range(len(lat_list_for_lon_compute)):
        temp_list.append(haversine((lat_list[lat],lon_list[lon]), (lat_list[lat],lon_list[lon+1]), unit=Unit.METERS)) # Haversine to compute the distance
    x_distance.append(temp_list)
# Transpose the dims
x_distance = np.array(x_distance).transpose(1,0)
# Compute y_distance (it is same for all grids)
y_distance = haversine((lat_list[1],40), (lat_list[2],40), unit=Unit.METERS)
# get area matrix for the world
area = x_distance * y_distance
# verify the outcome. The outcome equals to the surface area of the Earth (~= 5.1)
print(area.sum())
print(area.shape)
# save to local disk
np.save(file = save_path + "grid_area.npy",arr = area)

97 145 96
506225092484712.06
(96, 144)


In [6]:
# Construct transformation_matrix
transformation_matrix = []
for i in range(1800):
    transformation_matrix.append(area)
transformation_matrix = np.array(transformation_matrix)

In [7]:
burned_area = np.where(burned_area == burned_area.max(), 0, burned_area)
# Compute the burned area
burned_area = burned_area * 3600 * 30 * 24 * transformation_matrix
# print the dimension for the matrix
print(burned_area.shape)
# save to local disk
np.save(file = save_path + "burned_area.npy",arr = burned_area)

(1800, 96, 144)


### B. Elemforc

In [8]:
# 1. Import source file
elmforc = Dataset('../raw_dataset/elmforc.ssp5_hdm_0.5x0.5_simyr1850-2100_c190109.nc')
# 2. Get seq_dim for global time adjustments - Yearly Frequency
elmforc_seq_dim = np.array(num2date(elmforc["time"],elmforc["time"].units))

In [9]:
# 1.0 Elmforc Conversion (Strategy: propagate annual value to each month)
# Create an empty list to store index
seq_index = []
# Iterate time sequences to get index
for timepoint in surrogate_seq_dim:
    for i in range(0,len(elmforc_seq_dim)): # loop the elmforc seq to find out the time point greater than timepoint
        if timepoint < elmforc_seq_dim[i]:
            seq_index.append(i-1) # store the previous one's index in seq_index
            break

# create an empty list to store the feature
population_density_pro = []
# iterate the index list to get images and append in the list
for seq in seq_index:
    # A: Resizing - scaling to 96 * 144
    sub_graph = cv2.resize(elmforc["hdm"][seq], (sorrogate_size[1], sorrogate_size[0]))
    # B: append the transformed matrix to the list
    population_density_pro.append(sub_graph)
# convert the list to array
population_density = np.array(population_density_pro)
# print the shape 
print(population_density.shape)  # (1, 1800, 96, 144)

(1800, 96, 144)


In [10]:
# save to local disk
np.save(file = save_path + "human_density.npy",arr = population_density)

### C. Clmforc

In [11]:
# 1. Import source file
clmforc = Dataset('../raw_dataset/clmforc.Li_2012_climo1995-2011.T62.lnfm_Total_c140423.nc')
# 2. Get seq_dim for global time adjustments - Yearly Frequency
clmforc_seq_dim = np.array(num2date(clmforc["time"],clmforc["time"].units))
# 3. get the dim parameters for later resizing
clmforc_size = [clmforc["lnfm"][0].shape[0],clmforc["lnfm"][0].shape[1]]

#### Clmforc - Lightning Frequency Feature Engineering - working needed
- time adjustment
- resolution adjustment

In [12]:
# 2.0 Climatology Conversion (Strategy: split the dataset into 12 chunks and take the average for these matrix)
# Create an empty list to store index
seq_index = {}

for i in range(0,len(clmforc_seq_dim)):
    if i == 0:
        seq_index[clmforc_seq_dim[i].month] = [i]
    else:
        if clmforc_seq_dim[i-1].month != clmforc_seq_dim[i].month:
            seq_index[clmforc_seq_dim[i].month] = [i]
        else:
            seq_index[clmforc_seq_dim[i].month].append(i)

# Create the list to store the month average lightning frequency matrix
lightning_frequency_climatology_avg = []
for mon in seq_index.keys():
    temp_matrix = np.zeros((sorrogate_size[0], sorrogate_size[1]))
    for i in seq_index[mon]:
        # A: Resizing - scaling to 96 * 144
        sub_graph = cv2.resize(clmforc["lnfm"][i], (sorrogate_size[1], sorrogate_size[0]))
        # B: Padding Zeros to 224 * 224

        # C: append the transformed matrix to the list
        temp_matrix += sub_graph
    temp_matrix = temp_matrix / len(temp_matrix)
    lightning_frequency_climatology_avg.append(temp_matrix)

# Expand the list to the seq_index with corresponding months
lightning_frequency_climatology_avg_pro = []
for i in surrogate_seq_dim:
    lightning_frequency_climatology_avg_pro.append(lightning_frequency_climatology_avg[i.month-1])
lightning_frequency_climatology = np.array(lightning_frequency_climatology_avg_pro)
# print the shape 
print(lightning_frequency_climatology.shape)  # (1, 1800, 96, 144)

(1800, 96, 144)


In [13]:
# save to local disk
np.save(file = save_path + "light_frequency.npy",arr = lightning_frequency_climatology)

## 2. Feature Processing & Engineering - Image Data Generation

### A. Graph Conversion Function
- Convert the 2 dim matrix to 3 dim matrix (lat, lon, RGB channels)
- Magnifer the color change for compute vision learning

In [22]:
# === Function: single matrix conversion
def Graph_Conversion(data):
    # first gain the min value
    min_color = np.min(data)
    # set non-continental part to nan
    data = np.where(data == 1e36, min_color-1000000, data)
    # gain max value
    max_color = np.max(data)
    # set non-continental part to nan
    data = np.where(data == min_color-1000000, -5000000000, data)
    # compute color range
    color_range =  max_color - min_color + 1
    # copy the input data
    target = np.ones((96,144,3))
    
    # Iterations for each data point
    for loc, val in np.ndenumerate(data):
        r = (val - min_color) / color_range
        step = int(round(color_range/5))
        idx = int(r * 5)
        h = (idx + 1) * step + min_color
        m = idx * step + min_color
        local_r = (val - m) / (h-m)
        if h == m:
            local_r = 0
        else:
            None
        if val < min_color:
            target[loc] = np.array([0,0,0])
        if val > max_color:
            target[loc] = np.array([255, 255, 255])
        if idx == 0:
            target[loc] = np.array([0, int(local_r * 255), 255])
        if idx == 1:
            target[loc] = np.array([0, 255, int((1 - local_r) * 255)])
        if idx == 2:
            target[loc] = np.array([int(local_r * 255), 255, 0])
        if idx == 3:
            target[loc] = np.array([255, int((1 - local_r) * 255), 0])
        if idx == 4:
            target[loc] = np.array([255, 0, int(local_r * 255)])
            
    # return the converted matrix
    return target

In [23]:
# === Function: feature conversion (1800, 96, 144)
def Seq_to_Img(feature):
    feature_lst = []
    # gain the time dim
    time_dim = feature.shape[0]
    for i in range(0, time_dim):
        feature_lst.append(Graph_Conversion(feature[i]))
    feature_lst = np.array(feature_lst)
    print("-> Conversion Completed.")
    return feature_lst
# === Function: feature conversion (1800, 17, 96, 144)
def Seq_to_Img_Tree_Coverage(tc):
    feature_lst = []
    # gain the time dim
    time_dim = tc.shape[0]
    frame_dim = tc.shape[1]
    for i in range(0, time_dim):
        temp_lst = []
        for j in range(0, frame_dim):
            temp_lst.append(Graph_Conversion(tc[i][j]))
        feature_lst.append(temp_lst)
    feature_lst = np.array(feature_lst)
    print("-> Conversion Completed.")
    return feature_lst

### B. Feature Transformation from Sequence to RGB Image Matrix

In [24]:
# === Call functions to perform conversions
fuel_load_cwdc_img = Seq_to_Img(fuel_load_cwdc)
fuel_load_deadcrootc_img = Seq_to_Img(fuel_load_deadcrootc)
fuel_wetness_img = Seq_to_Img(fuel_wetness)
fuel_temperature_img = Seq_to_Img(fuel_temperature)
climate_wind_img = Seq_to_Img(climate_wind)
climate_tbot_img = Seq_to_Img(climate_tbot)
climate_rh2m_img = Seq_to_Img(climate_rh2m)
climate_rain_img = Seq_to_Img(climate_rain)
population_density_img = Seq_to_Img(population_density)
lightning_frequency_climatology_img = Seq_to_Img(lightning_frequency_climatology)
burned_area_img = Seq_to_Img(burned_area)
tree_coverage_img = Seq_to_Img_Tree_Coverage(tree_coverage)


-> Conversion Completed.
-> Conversion Completed.
-> Conversion Completed.
-> Conversion Completed.
-> Conversion Completed.
-> Conversion Completed.
-> Conversion Completed.


C:\Users\A\AppData\Local\Temp\ipykernel_27784\2694456574.py:23: RuntimeWarning: invalid value encountered in scalar divide
  local_r = (val - m) / (h-m)
C:\Users\A\AppData\Local\Temp\ipykernel_27784\2694456574.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  local_r = (val - m) / (h-m)


-> Conversion Completed.
-> Conversion Completed.
-> Conversion Completed.
-> Conversion Completed.


### C. Save to Local Disk (Img Matrix Dataset) & Feature Engineering Summary

In [25]:
# === Summary for Feature Engineering
feature_summary_panel = pd.DataFrame(columns = ["Feature","Category","Feature Array Dimensions (time, lat, lon)"])
# === add to dataframe
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["fuel_load_cwdc","Fuel",fuel_load_cwdc_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["fuel_load_deadcrootc","Fuel",fuel_load_deadcrootc_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["fuel_wetness","Fuel",fuel_wetness_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["fuel_temperature","Fuel",fuel_temperature_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["climate_wind","Climate",climate_wind_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["climate_tbot","Climate",climate_tbot_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["climate_rh2m","Climate",climate_rh2m_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["climate_rain","Climate",climate_rain_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["tree_coverage","Tree",tree_coverage_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["population_density","Population",population_density_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["lightning_frequency","Light",lightning_frequency_climatology_img.shape]
feature_summary_panel.loc[len(feature_summary_panel) + 1] = ["burned_area","Output",burned_area_img.shape]
# === show the result
feature_summary_panel

,Feature,Category,"Feature Array Dimensions (time, lat, lon)"
1,fuel_load_cwdc,Fuel,"(1800, 96, 144, 3)"
2,fuel_load_deadcrootc,Fuel,"(1800, 96, 144, 3)"
3,fuel_wetness,Fuel,"(1800, 96, 144, 3)"
4,fuel_temperature,Fuel,"(1800, 96, 144, 3)"
5,climate_wind,Climate,"(1800, 96, 144, 3)"
6,climate_tbot,Climate,"(1800, 96, 144, 3)"
7,climate_rh2m,Climate,"(1800, 96, 144, 3)"
8,climate_rain,Climate,"(1800, 96, 144, 3)"
9,population_density,Population,"(1800, 96, 144, 3)"
10,lightning_frequency,Light,"(1800, 96, 144, 3)"


In [26]:
# === define saving route
save_route = "../dataset/"

# === save files to local disk
np.save(file = save_route + "fuel_load_cwdc_img.npy",arr = fuel_load_cwdc_img)
np.save(file = save_route + "fuel_load_deadcrootc_img.npy",arr = fuel_load_deadcrootc_img)
np.save(file = save_route + "fuel_wetness_img.npy",arr = fuel_wetness_img)
np.save(file = save_route + "fuel_temperature_img.npy",arr = fuel_temperature_img)
np.save(file = save_route + "climate_wind_img.npy",arr = climate_wind_img)
np.save(file = save_route + "climate_tbot_img.npy",arr = climate_tbot_img)
np.save(file = save_route + "climate_rh2m_img.npy",arr = climate_rh2m_img)
np.save(file = save_route + "climate_rain_img.npy",arr = climate_rain_img)
np.save(file = save_route + "tree_coverage_img.npy",arr = tree_coverage_img)
np.save(file = save_route + "population_density_img.npy",arr = population_density_img)
np.save(file = save_route + "lightning_frequency_climatology_img.npy",arr = lightning_frequency_climatology_img)
np.save(file = save_route + "burned_area.npy",arr = burned_area)

## 3. Regional Segmentation Dictionary

### A. Import dataset and create dictionary

In [2]:
region_seg = np.array(Dataset('../raw_dataset/gfed_14regions.nc')["basic_14regions"])

In [3]:
region_dict = {"BONA":[],"TENA":[],"CEAM":[],"NHSA":[],"SHSA":[],
               "MIDE":[],"NHAF":[],"EURO":[],"SHAF":[],"BOAS":[],
               "CEAS":[],"SEAS":[],"EQAS":[],"AUST":[]}

In [4]:
for lat in range(region_seg.shape[0]):
    for lon in range(region_seg.shape[1]):
        if region_seg[lat,lon] == 1:
            region_dict["BONA"] = region_dict["BONA"]+[[lat,lon]]
        elif region_seg[lat,lon] == 2:
            region_dict["TENA"] = region_dict["TENA"]+[[lat,lon]]
        elif region_seg[lat,lon] == 3:
            region_dict["CEAM"] = region_dict["CEAM"]+[[lat,lon]]
        elif region_seg[lat,lon] == 4:
            region_dict["NHSA"] = region_dict["NHSA"]+[[lat,lon]]
        elif region_seg[lat,lon] == 5:
            region_dict["SHSA"] = region_dict["SHSA"]+[[lat,lon]]
        elif region_seg[lat,lon] == 6:
            region_dict["MIDE"] = region_dict["MIDE"]+[[lat,lon]]
        elif region_seg[lat,lon] == 7:
            region_dict["NHAF"] = region_dict["NHAF"]+[[lat,lon]]
        elif region_seg[lat,lon] == 8:
            region_dict["EURO"] = region_dict["EURO"]+[[lat,lon]]
        elif region_seg[lat,lon] == 9:
            region_dict["SHAF"] = region_dict["SHAF"]+[[lat,lon]]
        elif region_seg[lat,lon] == 10:
            region_dict["BOAS"] = region_dict["BOAS"]+[[lat,lon]]
        elif region_seg[lat,lon] == 11:
            region_dict["CEAS"] = region_dict["CEAS"]+[[lat,lon]]
        elif region_seg[lat,lon] == 12:
            region_dict["SEAS"] = region_dict["SEAS"]+[[lat,lon]]
        elif region_seg[lat,lon] == 13:
            region_dict["EQAS"] = region_dict["EQAS"]+[[lat,lon]]
        elif region_seg[lat,lon] == 14:
            region_dict["AUST"] = region_dict["AUST"]+[[lat,lon]]

### B. Save to local disk

In [5]:
with open("../dataset/region_segmentation.json","w", encoding='utf-8') as f:
    f.write(json.dumps(region_dict,ensure_ascii=False)) 